In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Load the training and test data
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [3]:
import pandas as pd

def preprocess_data(df):
    # Impute Age based on Pclass
    def impute_age(cols):
        Age = cols.iloc[0]
        Pclass = cols.iloc[1]
        
        if pd.isnull(Age):
            if Pclass == 1:
                return 37
            elif Pclass == 2:
                return 29
            else:
                return 24
        else:
            return Age

    df['Age'] = df[['Age', 'Pclass']].apply(impute_age, axis=1)
    
    # Drop Cabin and fill missing Fare values
    df.drop('Cabin', axis=1, inplace=True)
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    
    # Fill missing Embarked values
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    
    # Encode categorical variables
    # Ensure dummy variables are integers
    sex = pd.get_dummies(df['Sex'], drop_first=True).astype(int)
    embark = pd.get_dummies(df['Embarked'], drop_first=True).astype(int)

    
    df.drop(['Sex', 'Embarked', 'Name', 'Ticket'], axis=1, inplace=True)
    df = pd.concat([df, sex, embark], axis=1)
    
    return df

# Apply preprocessing to train and test DataFrames
train = preprocess_data(train)
test = preprocess_data(test)


In [4]:
test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,892,3,34.5,0,0,7.8292,1,1,0
1,893,3,47.0,1,0,7.0000,0,0,1
2,894,2,62.0,0,0,9.6875,1,1,0
3,895,3,27.0,0,0,8.6625,1,0,1
4,896,3,22.0,1,1,12.2875,0,0,1


In [5]:
train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,1,0,3,22.0,1,0,7.2500,1,0,1
1,2,1,1,38.0,1,0,71.2833,0,0,0
2,3,1,3,26.0,0,0,7.9250,0,0,1
3,4,1,1,35.0,1,0,53.1000,0,0,1
4,5,0,3,35.0,0,0,8.0500,1,0,1


In [6]:
X_train =train.drop('Survived',axis=1)
y_train =train['Survived']

In [7]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(max_iter=200)
logmodel.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

In [8]:
X_test_final = test

In [9]:
prediction = logmodel.predict(X_test_final)

In [10]:
output = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': prediction})

In [11]:
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [12]:
output.to_csv('submission.csv', index=False)